In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.dirname(os.getcwd()))
os.chdir('Data')

In [97]:
# load in all the data
od_deaths_fips = pd.read_csv('od_deaths_fips.csv')
unemployment_clean = pd.read_csv('unemployment_clean.csv')
opioid = pd.read_csv('opioid_dispense.csv')
eth_age = pd.read_csv('ethnicity_age.csv')
incar = pd.read_csv('incarceration.csv')
incar_new = pd.read_csv('incarceration_after2020.csv')
pov_inc = pd.read_csv('saipe_99_20.csv')

In [100]:
# clean: main data set
od_deaths = od_deaths_fips.drop('Unnamed: 0', axis = 1)

In [72]:
# cleaning: unemployment 
unemployment_clean = unemployment_clean.drop('State', axis = 1)
unemployment_clean.columns = ['County Code', 'County', 'Year', 'Unemployment_rate']
unemploy = unemployment_clean[['County Code', 'Year', 'Unemployment_rate']]

In [5]:
# cleaning: dispense rates 
dispenses = opioid[['Year', 'FIPS', 'dispenserate100']]
dispenses.columns = ['Year', 'County Code', 'Dispense_rate']

In [112]:
# cleaning: age/ethnicity
eth_age = eth_age.rename(columns = {'County_Code': 'County Code'})
eth_age['County Code'] = eth_age['County Code'].astype(int)

In [111]:
# cleaning: incarceration rates
incar_subset = incar[['year', 'fips', 'state', 'county_name', 'urbanicity', 'total_jail_pop', 'total_jail_pop_rate']]
incar_subset.columns = ['Year', 'County Code', 'State Name', 'County Name', 'Urbanicity', 'Jail Population', 'Incarceration Rate per 100k']
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
incar_subset.replace({'State Name': abbrev_to_us_state}, inplace = True)

In [88]:
# cleaning: incarceration rates pt. 2
incar_new['Year'] = pd.DatetimeIndex(incar_new['date']).year
incar_new2 = incar_new.drop('resident_population', axis = 1).\
                            rename(columns = {'fips': 'County Code', 'county_name': 'County Name',
                                              'state_name': 'State Name', 'jail_population': 'Jail Population',
                                             'jail_incarceration_rate_per_100k': 'Incarceration Rate per 100k',
                                             'urbanicity': 'Urbanicity'})
grouped_incar = incar_new2.groupby(['Year', 'County Code', 'County Name', 'Urbanicity', 'State Name'], as_index = False).mean()
grouped_incar = grouped_incar[['Year', 'County Code', 'State Name', 'County Name', 'Urbanicity', 'Jail Population', 'Incarceration Rate per 100k']]

incar_combined = pd.concat([incar_subset, grouped_incar])

In [98]:
# cleaning: poverty/income
pov_inc = pov_inc.drop('Unnamed: 0', axis = 1)
pov_inc = pov_inc.rename(columns = {'FIPS': 'County Code'})

In [115]:
# merging all the data
merge1 = od_deaths.merge(unemploy, on = ['County Code', 'Year'], how = 'left')
merge2 = merge1.merge(dispenses, on = ['County Code', 'Year'], how = 'left')
merge3 = merge2.merge(eth_age, on = ['County Code', 'Year'], how = 'left')
merge4 = merge3.merge(incar_combined, on = ['County Code', 'Year'], how = 'left')
merge5 = merge4.merge(pov_inc, on = ['County Code', 'Year'], how = 'left')

,Year,State_x,State Abv,State Code,County_x,County Code,Deaths,Population,Crude Rate,Cruder Rate,...,State Name,County Name,Urbanicity,Jail Population,Incarceration Rate per 100k,StateFIPS,CountyFIPS,PovertyCount,PovertyPercentage,MedianHHI
0,1999.0,Alabama,AL,1.0,"Baldwin County, AL",1003,11.0,137555.0,Unreliable,7.996801,...,Alabama,Baldwin County,small/mid,390.000000,440.340000,1,3,14668,10.5,39194
1,1999.0,Alabama,AL,1.0,"Jefferson County, AL",1073,34.0,662845.0,5.13,5.129404,...,Alabama,Jefferson County,urban,1779.000000,408.230000,1,73,89661,13.7,35885
2,1999.0,Alabama,AL,1.0,"Madison County, AL",1089,10.0,274693.0,Unreliable,3.640428,...,Alabama,Madison County,small/mid,713.000000,382.130000,1,89,30056,11.1,43718
3,1999.0,Alabama,AL,1.0,"Mobile County, AL",1097,28.0,399323.0,7.01,7.011868,...,Alabama,Mobile County,small/mid,1223.000000,470.210000,1,97,72372,18.3,32396
4,1999.0,Alaska,AK,2.0,"Anchorage Borough, AK",2020,21.0,259348.0,8.10,8.097228,...,Alaska,Anchorage Municipality,small/mid,NaN,NaN,2,20,18397,7.2,52959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,2020.0,Wisconsin,WI,55.0,"Winnebago County, WI",55139,38.0,171631.0,22.14,22.140522,...,NaN,NaN,NaN,NaN,NaN,55,139,14219,8.7,64653
15892,2020.0,Wisconsin,WI,55.0,"Wood County, WI",55141,18.0,72560.0,Unreliable,24.807056,...,NaN,NaN,NaN,NaN,NaN,55,141,6732,9.4,54154
15893,2020.0,Wyoming,WY,56.0,"Laramie County, WY",56021,17.0,100595.0,Unreliable,16.899448,...,NaN,NaN,NaN,NaN,NaN,56,21,7242,7.4,69450
15894,2020.0,Wyoming,WY,56.0,"Natrona County, WY",56025,16.0,80815.0,Unreliable,19.798305,...,NaN,NaN,NaN,NaN,NaN,56,25,7420,9.4,65901


In [7]:
# this is to add region to our data... if we want to that is, eventually

# northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania']
# midwest = ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota']
# south = ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'Washington, D.C.', 'West Virginia', 'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas', 'District of Co']
# west = ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
# merged2['Region'] = np.where(merged2['State'].isin(northeast), 'Northeast', 'none')
# merged2['Region'] = np.where(merged2['State'].isin(midwest), 'Midwest', merged2['Region'])
# merged2['Region'] = np.where(merged2['State'].isin(south), 'South', merged2['Region'])
# merged2['Region'] = np.where(merged2['State'].isin(west), 'West', merged2['Region'])

In [116]:
merge5.to_csv('od_deaths_merged_v2.csv', index = False)